In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [3]:
!kaggle competitions download -c udea-ai4eng-20242

 65% 13.0M/20.1M [00:00<00:00, 129MB/s]
100% 20.1M/20.1M [00:00<00:00, 139MB/s]


In [4]:
!unzip udea-ai4eng-20242.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [6]:
pip install fuzzywuzzy[speedup]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.1 MB/s eta 0:00:00


In [7]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [8]:
import pandas as pd
import re
from fuzzywuzzy import process

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

z = pd.read_csv('train.csv')
t = pd.read_csv('test.csv')

labels = z["RENDIMIENTO_GLOBAL"].values

dts_labels = np.random.choice(labels, 296786, replace=False)


source_cols = [i for i in z.columns if i!="RENDIMIENTO_GLOBAL"]
z = pd.concat((z[source_cols], t[source_cols]))
z.index = range(len(z))


mapBinary = {'Si': 1, 'S': 1, 'No': 0, 'N':0}
binaryColumns = ['FAMI_TIENEINTERNET' , 'ESTU_PAGOMATRICULAPROPIO']

for column in binaryColumns:
    z[column] = z[column].map(mapBinary)

variablesFaltantesBinaryCol = z[binaryColumns].isnull().sum()



# Definir las columnas de interés
columnas_de_interes = ['FAMI_TIENEINTERNET']
datos_imputados = z

# Iterar sobre las columnas de interés
for columna in columnas_de_interes:
    # Calcular la proporción de valores existentes
    proporciones = datos_imputados[columna].value_counts(normalize=True)

    # Calcular la proporción de 1 y 0
    prop_1 = proporciones.get(1, 0)
    prop_0 = proporciones.get(0, 0)

    # Imputar los valores faltantes de forma aleatoria
    datos_imputados[columna].fillna(np.random.choice([0, 1], p=[prop_0, prop_1]), inplace=True)
# Llenar los valores faltantes en la columna 'ESTU_PAGOMATRICULAPROPIO' con '0'

z['ESTU_PAGOMATRICULAPROPIO'].fillna(0, inplace=True)



# Diccionario para FAMI_ESTRATOVIVIENDA
mapeo_estrato = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
# Aplicar map a las columnas con el diccionario
z['FAMI_ESTRATOVIVIENDA'] = z['FAMI_ESTRATOVIVIENDA'].map(mapeo_estrato)

faltantesEstrato = z['FAMI_ESTRATOVIVIENDA'].isnull().sum()

# Calcular la media y la mediana de FAMI_ESTRATOVIVIENDA
media_estrato = z['FAMI_ESTRATOVIVIENDA'].mean()

# Imputar los valores faltantes con la media
z['FAMI_ESTRATOVIVIENDA'] = z['FAMI_ESTRATOVIVIENDA'].fillna(media_estrato)



z['FAMI_ESTRATOVIVIENDA'] = z['FAMI_ESTRATOVIVIENDA'].astype(int)



# Definir el mapeo de categorías a valores numéricos para cada columna
mapeo_valores_matricula = {
    'No pagó matrícula': 0,
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7
}

mapeo_valores_horas_trabajo = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}


# Convertir las columnas a formato numérico utilizando el mapeo
z['ESTU_VALORMATRICULAUNIVERSIDAD'] = z['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapeo_valores_matricula)
z['ESTU_HORASSEMANATRABAJA'] = z['ESTU_HORASSEMANATRABAJA'].map(mapeo_valores_horas_trabajo)


# Calcular la media total de la columna ESTU_VALORMATRICULAUNIVERSIDAD
media_valormatricula = z['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()

# Remplazar los valores faltantes en ESTU_VALORMATRICULAUNIVERSIDAD con la media total
z['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(media_valormatricula, inplace=True)

# Obtener los valores únicos en la columna ESTU_HORASSEMANATRABAJA
valores_horas_trabajo = z['ESTU_HORASSEMANATRABAJA'].dropna().unique()

# Calcular la cantidad de valores faltantes en ESTU_HORASSEMANATRABAJA
num_valores_faltantes = z['ESTU_HORASSEMANATRABAJA'].isnull().sum()


media_trabaja = z['ESTU_HORASSEMANATRABAJA'].mean()
z['ESTU_HORASSEMANATRABAJA'].fillna(media_trabaja, inplace=True)



# Eliminar de los valores de las columnas FAMI_EDUCACIONPADRE Y FAMI_EDUCACIONMADRE las palabras "Incompleto", "completo",
def limpiar_cadena(cadena):
  if isinstance(cadena, str):
    cadena = cadena.replace("incompleta", "").replace("incompleto", "").replace("completo", "").replace("completa", "").strip()
  return cadena

# Aplicar la función a las columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
z['FAMI_EDUCACIONPADRE'] = z['FAMI_EDUCACIONPADRE'].apply(limpiar_cadena)
z['FAMI_EDUCACIONMADRE'] = z['FAMI_EDUCACIONMADRE'].apply(limpiar_cadena)



# Obtener los valores más frecuentes para cada columna
top_padre = z['FAMI_EDUCACIONPADRE'].mode()[0]
top_madre = z['FAMI_EDUCACIONMADRE'].mode()[0]

# Rellenar los valores faltantes con los valores más frecuentes
z['FAMI_EDUCACIONPADRE'].fillna(top_padre, inplace=True)
z['FAMI_EDUCACIONMADRE'].fillna(top_madre, inplace=True)


# Calcula la moda para las columnas seleccionadas
moda_padre = z['FAMI_EDUCACIONPADRE'].mode()[0]
moda_madre = z['FAMI_EDUCACIONMADRE'].mode()[0]

# Rellena los valores faltantes con la moda
z['FAMI_EDUCACIONPADRE'].fillna(moda_padre, inplace=True)
z['FAMI_EDUCACIONMADRE'].fillna(moda_madre, inplace=True)



# Aplicar one-hot encoding a las columnas 'FAMI_EDUCACIONPADRE' y 'FAMI_EDUCACIONMADRE'
z = pd.get_dummies(z, columns=['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], prefix=['PADRE_EDUCACION', 'MADRE_EDUCACION'])

# Asignar 0 para False y 1 para True en las columnas generadas por one-hot encoding
z.replace({True: 1, False: 0}, inplace=True)

# Lista de columnas que se desean convertir de float a int
columnas_float_a_entero = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
                           'FAMI_TIENEINTERNET',
                           'ESTU_PAGOMATRICULAPROPIO', 'FAMI_ESTRATOVIVIENDA']

# Convertir las columnas de float a enteros
z[columnas_float_a_entero] = z[columnas_float_a_entero].astype(int)



departamentosRegion = {
    'Caribe': ['ATLANTICO', 'BOLIVAR', 'CORDOBA', 'CESAR' 'LA GUAJIRA', 'MAGDALENA', 'SUCRE', 'SAN ANDRES'],
    'Andina': ['ANTIOQUIA', 'BOYACA', 'CUNDINAMARCA', 'RISARALDA', 'TOLIMA', 'CALDAS', 'CAUCA', 'HUILA', 'NORTE SANTANDER', 'QUINDIO', 'SANTANDER', 'BOGOTA'],
    'Pacifica': ['CHOCO', 'NARIÑO', 'VALLE'],
    'Orinoquia': ['ARAUCA', 'CASANARE', 'META'],
    'Amazonica': ['AMAZONAS', 'CAQUETA', 'GUAVIARE', 'VAUPES', 'PUTUMAYO'],
}
# Prefijo para los nombres de las nuevas columnas
prefijo_columnas = 'DEPARTAMENTO_'

# Crear columnas para cada región y asignar los departamentos correspondientes
for region, departamentos in departamentosRegion.items():
    z[prefijo_columnas + region.replace(' ', '_').upper()] = z['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: x if x in departamentos else None)

# Eliminar la columna original ESTU_PRGM_DEPARTAMENTO
z.drop(columns=['ESTU_PRGM_DEPARTAMENTO'], inplace=True)



map_recurrencia_caribe = {'SAN ANDRES': 1, 'LA GUAJIRA': 2, 'CESAR': 3, 'SUCRE': 4, 'MAGDALENA': 5, 'CORDOBA': 6, 'BOLIVAR': 7, 'NORTE SANTANDER': 8, 'ATLANTICO': 9}
map_recurrencia_andina = {'QUINDIO': 1, 'HUILA': 2, 'CAUCA': 3, 'CALDAS': 4, 'TOLIMA': 5, 'RISARALDA': 6, 'CUNDINAMARCA': 7, 'BOYACA': 8, 'SANTANDER': 9, 'ANTIOQUIA': 10, 'BOGOTA': 11}
map_recurrencia_pacifica = {'CHOCO': 1, 'NARIÑO': 2, 'VALLE': 3}
map_recurrencia_orinoquia = {'ARAUCA': 1, 'CASANARE': 2, 'META': 3}
map_recurrencia_amazonica = {'VAUPES': 1, 'GUAVIARE': 2, 'AMAZONAS': 3, 'PUTUMAYO': 4, 'CAQUETA': 5}


z['DEPARTAMENTO_CARIBE'] = z['DEPARTAMENTO_CARIBE'].map(map_recurrencia_caribe)
z['DEPARTAMENTO_ANDINA'] = z['DEPARTAMENTO_ANDINA'].map(map_recurrencia_andina)
z['DEPARTAMENTO_PACIFICA'] = z['DEPARTAMENTO_PACIFICA'].map(map_recurrencia_pacifica)
z['DEPARTAMENTO_ORINOQUIA'] = z['DEPARTAMENTO_ORINOQUIA'].map(map_recurrencia_orinoquia)
z['DEPARTAMENTO_AMAZONICA'] = z['DEPARTAMENTO_AMAZONICA'].map(map_recurrencia_amazonica)

columnas_reemplazar_departamentos = ['DEPARTAMENTO_CARIBE', 'DEPARTAMENTO_ANDINA', 'DEPARTAMENTO_PACIFICA', 'DEPARTAMENTO_ORINOQUIA', 'DEPARTAMENTO_AMAZONICA']
z[columnas_reemplazar_departamentos] = z[columnas_reemplazar_departamentos].fillna(0).astype(int)



# Función para normalizar el texto
def normalize_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Normalizar la columna 'ESTU_PRGM_ACADEMICO'
z['ESTU_PRGM_ACADEMICO_NORMALIZADO'] = z['ESTU_PRGM_ACADEMICO'].apply(normalize_text)

# Diccionario para almacenar mapeos
text_map = {}

# Iterar sobre cada entrada única en la columna normalizada
for text in z['ESTU_PRGM_ACADEMICO_NORMALIZADO'].unique():
    match = process.extractOne(text, text_map.keys(), score_cutoff=90)

    if match:  # Verificamos que haya una coincidencia válida
        best_match, score = match  # Desempaquetamos solo si existe
        text_map[text] = text_map[best_match]
    else:
        # Si no hay match, el texto es considerado una nueva entrada
        text_map[text] = text

# Aplicar el mapeo de agrupación al DataFrame
z['ESTU_PRGM_ACADEMICO_LIMPIO'] = z['ESTU_PRGM_ACADEMICO_NORMALIZADO'].map(text_map)



# Lista de palabras irrelevantes (stop words) que queremos eliminar
stop_words = [
    'profesional', 'universitario', 'en', 'la',
    'con', 'enfasis', 'para', 'industria', 'basica'
]

# Función para eliminar palabras irrelevantes
def remove_stop_words(text):
    # Dividimos el texto en palabras y eliminamos las que coincidan con las stop words
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Aplicamos la función para eliminar términos irrelevantes
z['ESTU_PRGM_ACADEMICO_REDUCIDO'] = z['ESTU_PRGM_ACADEMICO_LIMPIO'].apply(remove_stop_words)

# Diccionario para almacenar mapeos
text_map = {}

# Iterar sobre cada entrada única en la columna reducida
for text in z['ESTU_PRGM_ACADEMICO_REDUCIDO'].unique():
    # Usar fuzzywuzzy para encontrar el mejor match con un umbral de similitud
    match = process.extractOne(text, text_map.keys(), score_cutoff=85)

    if match:  # Verificamos que match no sea None
        best_match, score = match  # Desempaquetamos la tupla solo si match es válido
        text_map[text] = text_map[best_match]  # Asignar la entrada al grupo ya existente
    else:
        # Si no hay match, agregarlo como nueva entrada en el mapa
        text_map[text] = text

# Crear una columna con las entradas agrupadas
z['ESTU_PRGM_ACADEMICO_FINAL'] = z['ESTU_PRGM_ACADEMICO_REDUCIDO'].map(text_map)




# Diccionario de categorías
categorias = {
    "Ciencias de la Salud": [
        "enfermeria", "psicologia", "medicina veterinaria", "odontologia", "quimica farmaceutica",
        "instrumentacion quirurgica", "fisioterapia", "farmacia", "optometria", "bacteriologia",
        "terapia respiratoria", "fonoaudiologia"
    ],
    "Ingenierías": [
        "ingenieria mecanica", "ingenieria industrial", "ingenieria civil", "ingeniera multimedia"
    ],
    "Administración y Negocios": [
        "administracion de empresas", "administracin salud ocupacional", "administracin financiera",
        "hoteleria y turismo", "negocios internacionales", "agronegocios", "administracion de calidad",
        "administracion servicio"
    ],
    "Ciencias Sociales y Humanidades": [
        "derecho", "comunicacion social", "ciencia politica", "trabajo social", "sociologia",
        "relaciones internacionales", "jurisprudencia", "estudios literarios", "historia",
        "licenciatura ciencias sociales", "licenciatura pedagogia infantil", "licenciatura educacion humanidadesingles"
    ],
    "Artes y Diseño": [
        "maestro msica", "comunicacion audiovisual", "diseo industrial", "diseo grafico",
        "diseo visual", "arte danzario", "artes plasticas"
    ],
    "Ciencias Exactas y Naturales": [
        "estadistica", "zootecnia", "quimica farmaceutica", "agronomia", "geologia"
    ],
    "Otras": [
        "mercadeo y publicidad", "arquitectura", "economia", "criminalistica", "entrenamiento deportivo",
        "traduccion inglesfrancesespaol", "antropologia"
    ]
}

# Crear un diccionario inverso para mapear programas individuales a su categoría
mapeo_programas = {programa: categoria for categoria, programas in categorias.items() for programa in programas}

# Función para mapear cada programa académico a su categoría
z['CATEGORIA_PRGM_ACADEMICO'] = z['ESTU_PRGM_ACADEMICO_FINAL'].map(mapeo_programas)



columns_to_drop = ['ESTU_PRGM_ACADEMICO_NORMALIZADO', 'ESTU_PRGM_ACADEMICO_LIMPIO', 'ESTU_PRGM_ACADEMICO_REDUCIDO', 'ESTU_PRGM_ACADEMICO_FINAL', 'ESTU_PRGM_ACADEMICO']
z = z.drop(columns=columns_to_drop)


z = pd.get_dummies(z, columns=['CATEGORIA_PRGM_ACADEMICO'], prefix=['PRGM_ACADEMICO'])


z.replace({True: 1, False: 0}, inplace=True)



columnas_float_a_entero = z.select_dtypes(include=['float64']).columns.tolist()

# Convertir las columnas de float a enteros
z[columnas_float_a_entero] = z[columnas_float_a_entero].astype(int)


originalData = pd.read_csv('train.csv')
lentr = len(originalData)

Xtrk, ytrk = z.iloc[:lentr].values, originalData["RENDIMIENTO_GLOBAL"].values
Xtsk  = z.iloc[lentr:].values



n = int(len(Xtrk)*0.8)





<ipython-input-8-ec8388e8ab8e>:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_imputados[columna].fillna(np.random.choice([0, 1], p=[prop_0, prop_1]), inplace=True)
<ipython-input-8-ec8388e8ab8e>:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[

In [ ]:
idxs = np.random.permutation(len(Xtrk))
idxs_trm = idxs[:n]
idxs_tsm = idxs[n:]

Xtrm = Xtrk[idxs_trm]
ytrm = ytrk[idxs_trm]

Xtsm = Xtrk[idxs_tsm]
ytsm = ytrk[idxs_tsm]



train_pool = Pool(Xtrm, ytrm)

model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=10,
    loss_function='MultiClass',
    verbose=100
)

model.fit(train_pool, early_stopping_rounds=50)

# Hacer predicciones
y_pred = model.predict(Xtsk)

accuracy = accuracy_score(dts_labels, y_pred)

0:	learn: 1.3657115	total: 1.54s	remaining: 25m 39s
100:	learn: 1.2228481	total: 1m 22s	remaining: 12m 15s
200:	learn: 1.2039223	total: 2m 40s	remaining: 10m 37s
300:	learn: 1.1873996	total: 4m 3s	remaining: 9m 25s
400:	learn: 1.1710085	total: 5m 22s	remaining: 8m 2s


In [ ]:
submissiondf = t[["ID"]]
submissiondf['RENDIMIENTO_GLOBAL'] = y_pred.flatten()


submissiondf.to_csv('Entregakaggle.csv', index=False)



In [ ]:
!head Entregakaggle.csv

ID,RENDIMIENTO_GLOBAL
550236,alto
98545,alto
499179,alto
782980,bajo
785185,bajo
58495,medio-bajo
705444,alto
557548,alto
519909,bajo
